# Train Baseline DeepFM on Google Colab

This notebook trains the baseline DeepFM model on normalized Avazu x4 dataset.

## Setup Instructions:
1. Upload this notebook to Google Colab
2. Runtime > Change runtime type > GPU (T4 GPU recommended)
3. Run all cells in order

## What this does:
- Clones your GitHub repository
- Installs dependencies
- Downloads normalized Avazu x4 dataset (or uses uploaded data)
- Trains DeepFM baseline model
- Downloads the trained checkpoint

## Step 1: Check GPU Availability

In [ ]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    print("GPU Memory:", torch.cuda.get_device_properties(0).total_memory / 1e9, "GB")
else:
    print("⚠️ No GPU detected. Go to Runtime > Change runtime type > GPU")

## Step 2: Clone Repository (Option A) or Upload Files (Option B)

### Option A: Clone from GitHub (Recommended)

In [ ]:
# Clone your repository
!git clone https://github.com/YOUR_USERNAME/SITxShopee_Recommender.git
%cd SITxShopee_Recommender

### Option B: Upload as ZIP file

If your repo is private or you prefer to upload:
1. Zip your entire project folder (exclude `data/` directory - too large)
2. Upload the ZIP file using the file browser on the left
3. Run the cell below to extract it

In [ ]:
# Uncomment if using ZIP upload method
# !unzip SITxShopee_Recommender.zip
# %cd SITxShopee_Recommender

## Step 3: Install Dependencies

In [ ]:
# Install required packages
!pip install -q scikit-learn pandas numpy pyyaml h5py tqdm transformers

# Verify installation
import sklearn
import pandas as pd
import numpy as np
import yaml
print("✓ All dependencies installed")

## Step 4: Prepare Dataset

### Option A: Download from Google Drive (Recommended)

**Before running Colab**: Upload your normalized dataset to Google Drive
1. Compress `data/Avazu/avazu_x4_normalized/` into a ZIP file
2. Upload to Google Drive
3. Get shareable link and extract file ID
4. Replace FILE_ID below

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Option A1: If dataset is in your Google Drive
# Copy from Drive to Colab workspace
!mkdir -p data/Avazu/
!cp -r "/content/drive/MyDrive/avazu_x4_normalized" data/Avazu/

# Verify
!ls -lh data/Avazu/avazu_x4_normalized/

### Option B: Download from URL

If you have the dataset hosted somewhere (e.g., Dropbox, Google Drive public link):

In [ ]:
# Uncomment and modify if downloading from URL
# !mkdir -p data/Avazu/
# !wget -O avazu_x4_normalized.zip "YOUR_DOWNLOAD_URL_HERE"
# !unzip avazu_x4_normalized.zip -d data/Avazu/
# !rm avazu_x4_normalized.zip

### Option C: Run normalization script on Colab

Download raw data and normalize on Colab (takes longer but works):

In [ ]:
# Uncomment to download and normalize on Colab
# !bash 1.prepare.sh  # Download raw Avazu datasets
# !python normalize_avazu_datasets.py  # Normalize them

## Step 5: Verify Setup

In [ ]:
# Check if everything is ready
import os

checks = [
    ("FuxiCTR framework", os.path.exists("fuxictr")),
    ("DeepFM model", os.path.exists("model_zoo/DeepFM/src/DeepFM.py")),
    ("Model config", os.path.exists("model_zoo/DeepFM/config/model_config.yaml")),
    ("Dataset config", os.path.exists("model_zoo/DeepFM/config/dataset_config.yaml")),
    ("Training data", os.path.exists("data/Avazu/avazu_x4_normalized/train.csv")),
    ("Validation data", os.path.exists("data/Avazu/avazu_x4_normalized/valid.csv")),
    ("Test data", os.path.exists("data/Avazu/avazu_x4_normalized/test.csv")),
]

all_ready = True
for name, exists in checks:
    status = "✓" if exists else "✗"
    print(f"{status} {name}")
    if not exists:
        all_ready = False

if all_ready:
    print("\n✅ All checks passed! Ready to train.")
else:
    print("\n⚠️ Some files are missing. Please check above.")

## Step 6: Start Training

This will train for 100 epochs. You can reduce epochs in the config if needed.

In [ ]:
# Train the model
!python model_zoo/DeepFM/run_expid.py \
    --expid DeepFM_avazu_normalized \
    --gpu 0

## Step 7: Check Training Results

In [ ]:
# List checkpoint files
!find model_zoo/DeepFM/Avazu/DeepFM_avazu_normalized/ -type f

# Show training log summary
!tail -50 model_zoo/DeepFM/Avazu/DeepFM_avazu_normalized/*/DeepFM_avazu_normalized.log

## Step 8: Download Checkpoint to Your Computer

Compress and download the trained checkpoint.

In [ ]:
# Compress checkpoint directory
!cd model_zoo/DeepFM/Avazu/DeepFM_avazu_normalized/ && \
    tar -czf /content/checkpoint_baseline.tar.gz .

print("✓ Checkpoint compressed to: /content/checkpoint_baseline.tar.gz")
print("\nDownload it using the file browser on the left, or:")

# Download via Colab files API
from google.colab import files
files.download('/content/checkpoint_baseline.tar.gz')

## Step 9: Save to Google Drive (Optional)

Save checkpoint to your Google Drive for easy access later.

In [ ]:
# Copy checkpoint to Google Drive
!cp /content/checkpoint_baseline.tar.gz "/content/drive/MyDrive/"
print("✓ Checkpoint saved to Google Drive")

## Notes:

### Expected Training Time:
- **With T4 GPU**: ~4-6 hours for 100 epochs on Avazu x4
- **With CPU**: Very slow (not recommended)

### To Reduce Training Time:
Edit `model_zoo/DeepFM/config/model_config.yaml` and change:
```yaml
DeepFM_avazu_normalized:
    epochs: 20  # Reduced from 100
```

### Monitoring Training:
Watch the AUC and logloss metrics. Training is working if:
- AUC increases over epochs
- Logloss decreases over epochs

### After Training:
1. Download the checkpoint
2. Extract on your local machine: `tar -xzf checkpoint_baseline.tar.gz`
3. Place in: `model_zoo/DeepFM/Avazu/DeepFM_avazu_normalized/`
4. Ready for Phase 2 (LLM-CTR training)!